# Deploying Machine Learning Models on GCP Kubernetes

<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequisites
* You need a running GCP cluster with kubernetes>1.8 with kubectl configured to use.
* If you wish to test the JSON schema checks you will need presently to enbale "alpha features" for your cluster (Jan 2018).
* A clone of the latest seldon core
- [python grpc tools](https://grpc.io/docs/quickstart/python.html)

## Install helm

In [1]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


## Start Seldon-Core CRD

In [2]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Mar  3 08:54:09 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s


NOTES:
NOTES: TODO




## Cordon off loadtest node

In [3]:
!kubectl get nodes

NAME                                      STATUS    ROLES     AGE       VERSION
gke-loadtest-default-pool-b0c49eff-4l6d   Ready     <none>    22m       v1.9.3-gke.0
gke-loadtest-default-pool-b0c49eff-gn9z   Ready     <none>    22m       v1.9.3-gke.0
gke-loadtest-default-pool-b0c49eff-m3g5   Ready     <none>    21m       v1.9.3-gke.0
gke-loadtest-default-pool-b0c49eff-vxsp   Ready     <none>    22m       v1.9.3-gke.0


In [4]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node "gke-loadtest-default-pool-b0c49eff-4l6d" cordoned
node "gke-loadtest-default-pool-b0c49eff-gn9z" cordoned
node "gke-loadtest-default-pool-b0c49eff-m3g5" cordoned


In [69]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[3].metadata.name}') role=locust

error: 'role' already has a value (locust), and --overwrite is false
error: 'role' already has a value (locust), and --overwrite is false
error: 'role' already has a value (locust), and --overwrite is false
node "gke-loadtest-default-pool-b0c49eff-n6h3" labeled


## Start seldon-core

In [6]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set cluster_manager.rbac=true \
        --set apife.enabled=true \
        --set engine.image.tag=0.1.6_SNAPSHOT_loadtest \
        --set cluster_manager.image.tag=0.1.6_SNAPSHOT_loadtest
        

NAME:   seldon-core
LAST DEPLOYED: Sat Mar  3 08:54:48 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-apiserver        1        0        0           0          1s
seldon-cluster-manager  1        0        0           0          1s
redis                   1        1        1           0          1s

==> v1/Service
NAME              TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)                        AGE
seldon-apiserver  NodePort   10.3.249.116  <none>       8080:30212/TCP,5000:31226/TCP  1s
redis             ClusterIP  10.3.249.106  <none>       6379/TCP                       1s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
redis-df886d999-f8tnb  0/1    ContainerCreating  0         1s


NOTES:
NOTES: TODO




In [8]:
!kubectl get pods -o wide

NAME                                      READY     STATUS    RESTARTS   AGE       IP         NODE
redis-df886d999-f8tnb                     1/1       Running   0          34s       10.0.1.7   gke-loadtest-default-pool-b0c49eff-vxsp
seldon-apiserver-64ccd4c5f4-p6dtq         1/1       Running   0          33s       10.0.1.8   gke-loadtest-default-pool-b0c49eff-vxsp
seldon-cluster-manager-68c8c6b5bf-bd2dd   1/1       Running   0          33s       10.0.1.9   gke-loadtest-default-pool-b0c49eff-vxsp


## Set up REST and GRPC methods

Port foward for testing
```bash
kubectl port-forward $(kubectl get pods -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') 8080:8080
```

In [9]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Setup python code to do REST and gRPC requests. **Only run this when the LoadBalancer created by GCP for the seldon-apife is running**

In [10]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
try:
    from commands import getoutput # python 2
except ImportError:
    from subprocess import getoutput # python 3

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://0.0.0.0:8080/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://0.0.0.0:8080/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print(response.text)
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("0.0.0.0:5000")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)


## Normal Operation

### Create Seldon Deployment

In [77]:
!kubectl apply -f resources/loadtest_simple_model.json

seldondeployment "seldon-core-loadtest" created


In [12]:
!kubectl get seldondeployments seldon-core-loadtest -o jsonpath='{.status}'

### Get Predictions

In [13]:
# REST Request
rest_request()

{
  "status": {
    "code": 0,
    "info": "",
    "reason": "",
    "status": "SUCCESS"
  },
  "meta": {
    "puid": "n6j7ef9anrkv3vp4gumq9gcgcv",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["class0", "class1", "class2"],
    "tensor": {
      "shape": [1, 3],
      "values": [0.1, 0.9, 0.5]
    }
  }
}


In [ ]:
# GRPC Request
grpc_request()

## Load test

In [14]:
!kubectl create clusterrolebinding default-admin --clusterrole=cluster-admin --serviceaccount=default:default

clusterrolebinding "default-admin" created


In [ ]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false

In [78]:
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[3].metadata.name}')

node "gke-loadtest-default-pool-b0c49eff-4l6d" uncordoned
node "gke-loadtest-default-pool-b0c49eff-gn9z" uncordoned
node "gke-loadtest-default-pool-b0c49eff-m3g5" uncordoned
node "gke-loadtest-default-pool-b0c49eff-n6h3" uncordoned


In [79]:
!helm install ../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://loadtest:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=32

NAME:   loadtest
LAST DEPLOYED: Sat Mar  3 14:57:09 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME             TYPE      CLUSTER-IP    EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.3.252.195  <none>       5557:31232/TCP,5558:30085/TCP,8089:31560/TCP  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-25h8b   0/1    ContainerCreating  0         1s
locust-slave-1-2krvc   0/1    Pending            0         1s
locust-slave-1-44lt5   0/1    Pending            0         1s
locust-slave-1-4h9cz   0/1    ContainerCreating  0         1s
locust-slave-1-4lk9f   0/1    Pending            0         1s
locust-slave-1-588hb   0/1    ContainerCreating  0         1s
locust-slave-1-5tmqj   0/1    ContainerCreating  0         1s
locust-slave-1-6dsdt   0/1    Pending            0         1s
locust-slave-1-6rn69   0/1    Pending            0         0s
locust-slave-1-7m6vm   0/1    P

In [76]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[3].metadata.name}')

node "gke-loadtest-default-pool-b0c49eff-4l6d" cordoned
node "gke-loadtest-default-pool-b0c49eff-gn9z" cordoned
node "gke-loadtest-default-pool-b0c49eff-m3g5" cordoned
node "gke-loadtest-default-pool-b0c49eff-n6h3" cordoned


In [80]:
!helm delete loadtest --purge

release "loadtest" deleted


## Update Deployment with Canary

In [ ]:
!kubectl apply -f resources/model_with_canary.json -n seldon

Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

In [ ]:
rest_request()

In [ ]:
grpc_request()

## Tear Down

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge